# NOC processing
#### This is the fourth and final step of the enrichment calculation
Calculate NOC proportion and append to the enrichment table  

Key output files
- `yyyy-mm-dd_enrichment_table_NOC_prop.csv`: enrichment values and NOC proportion  
- `yyyy-mm-dd_volcano_table.csv`: enrichment values and p-values  
  
both files are in the directory `output/enrichment_and_volcano_tables`

In [1]:
import os
import sys
from datetime import datetime
from pathlib import Path
import matplotlib.pyplot as plt
import pandas as pd

script_path = Path.cwd().parent.parent / "script"
data_path = Path.cwd().parent.parent / "data"
sys.path.append(str(script_path))

from pyseus import basic_processing as ip
from pyseus import contrast_tools as ct

### Load imputed NOC/Bulk table
The correct datestamp is required to find the input files

In [2]:
%store -r timestamp USE_FROZEN
if USE_FROZEN:
    raise Exception("USE_FROZEN is true, you probably want to skip enrichment and proceed from Fig1")
print(f"Timestamp: {timestamp}")
outprefix = f"{timestamp}_QC_filter_impute"

outdir = Path.cwd() / "output"
preprocessing_out = outdir / f"preprocessing"

NOC_path = preprocessing_out / f"{outprefix}_filtered_table.csv"

try:
    noc_table = pd.read_csv(NOC_path, header=[0], index_col=0)
except FileNotFoundError:
    print(f"File {NOC_path} not found.\nPlease rerun the previous steps or specify the correct timestamp, current value is {timestamp}")
except pd.errors.ParserError:
    print(f"There was an error parsing the CSV file at {NOC_path}.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

Timestamp: 2023-12-04


### Calculation of proportion

In [3]:
# take only the noc columns
NOC_kw = ["Cytosol", "Nuclear", "Organelle"]
noc_cols = [
    col for col in list(noc_table)
    if (any(kw in col for kw in NOC_kw)) and "Proteome" not in col
]
print(noc_cols)

['UnInfected_Cytosol_1', 'UnInfected_Cytosol_2', 'UnInfected_Cytosol_3', 'UnInfected_Nuclear_1', 'UnInfected_Nuclear_2', 'UnInfected_Nuclear_3', 'UnInfected_Organelle_1', 'UnInfected_Organelle_2', 'UnInfected_Organelle_3']


In [4]:
# just select the noc_cols + Protein IDs for merge purpose
noc_table = noc_table[['Protein IDs'] + noc_cols].copy()

In [5]:
# group the tables and take the median

# start RawTables class from basic processing to group the samples
noc_process = ip.RawTables(proteingroup=None, file_designated=True,
    info_cols=['Protein IDs'], sample_cols=noc_cols)

# designate the pre-computed noc filtered table on this class
noc_process.filtered_table = noc_table

# transform the intensity to linear space
#noc_process.transform_intensities(func=np.exp2)

# group replicates
noc_process.group_replicates(reg_exp=r'(.*)_\d+$')

# calculate the median
noc_medians = ip.median_replicates(noc_process.grouped_table)
noc_medians.head()

Intensity values have not been transformed yet from filtered table,
we recommend using transform_intensities() method before grouping replicates.

Using filtered_table to group replicates.


,UnInfected_Nuclear,UnInfected_Organelle,UnInfected_Cytosol,Protein IDs
2,166520000.0,76811000.0,75060000.0,A0A023T6R1;Q96A72;F5H6P7;F5H6N1;F5H3U9;F5H124
3,238020000.0,194700000.0,245940000.0,Q9Y5S9;A0A023T787;A0A0J9YW13
5,22978000.0,51652000.0,4330700.0,A0A0C4DFM1;A0A024QYR3;Q92544;B4DH88;B4DKC1;Q6Z...
6,0.0,0.0,3272600.0,A0A024QYR6;A0A1V0DNR7;A0A6G6A825;F6KD02;F6KD01...
7,24210000.0,77290000.0,9258400.0,Q99805;A0A024QYR8;B3KSG9


In [6]:
# we then take the sum of each column 
noc_medians['noc_sum_uninf'] = noc_medians['UnInfected_Cytosol'] + noc_medians['UnInfected_Organelle'] + noc_medians['UnInfected_Nuclear']

# remove all proteingroups where both inf and uninf noc_sum is 0 -> there is no information on these
noc_medians = noc_medians[(noc_medians['noc_sum_uninf'] > 0)].reset_index(drop=True)

# create a new dataframe with just the proportion: intensity divided by the sum
noc_proportion = noc_medians[['Protein IDs']].copy()
noc_proportion['NOC_cytosol'] = noc_medians['UnInfected_Cytosol'] / noc_medians['noc_sum_uninf']
noc_proportion['NOC_organelle'] = noc_medians['UnInfected_Organelle'] / noc_medians['noc_sum_uninf']
noc_proportion['NOC_nuclear'] = noc_medians['UnInfected_Nuclear'] / noc_medians['noc_sum_uninf']


### Merge NOC proportion with IP enrichment

In [7]:
# merge with the enrichment table
# read the enrichment table
enrich_out_dir = outdir / "enrichment_and_volcano_tables"
enrichment_csv_path = enrich_out_dir / f"{timestamp}_enrichment_table.csv"
enrichments = pd.read_csv(enrichment_csv_path, header=[0, 1], index_col=0)
meta_cols = enrichments["metadata"].columns
enrichments = enrichments.droplevel(0, axis=1)

In [8]:
# finally, merge with the enrichment table
enrichment_nocs = enrichments.merge(noc_proportion, on="Protein IDs", how="inner")

# standard formatting
enrichment_nocs = ct.standard_pyseus_headers(enrichment_nocs, meta_cols=meta_cols)

In [9]:
# save the enrichment table with NOC proportions
enrichment_NOC_csv_path = enrich_out_dir / f"{timestamp}_enrichment_table_NOC_prop.csv"
enrichment_nocs.to_csv(enrichment_NOC_csv_path)

In [10]:
# enrichment table with NOC proportions
enrichment_nocs

metadata  \
                                            Protein IDs   
0         A0A023T6R1;Q96A72;F5H6P7;F5H6N1;F5H3U9;F5H124   
1                          Q9Y5S9;A0A023T787;A0A0J9YW13   
2     A0A0C4DFM1;A0A024QYR3;Q92544;B4DH88;B4DKC1;Q6Z...   
3     A0A024QYR6;A0A1V0DNR7;A0A6G6A825;F6KD02;F6KD01...   
4                              Q99805;A0A024QYR8;B3KSG9   
...                                                 ...   
8536                                             X5D7P8   
8537                                             X5D8X9   
8538  X5DQV1;X5DNI1;B3KV96;E9PD68;B3KXQ5;Q14194;B3KT...   
8539                                             X5DQZ7   
8540                                             X6RHX1   

                                                                          \
                                   Majority protein IDs       Gene names   
0                       A0A023T6R1;Q96A72;F5H6P7;F5H6N1  FLJ10292;MAGOHB   
1                                     Q9Y5S9;A0A023T787       RBM8A;RBM8   
2     A0A0C4DFM1;A0A024QYR3;Q92544;B4DH88;B4DKC1;Q6ZTK5           TM9SF4   
3     A0A024QYR6;A0A1V0DNR7;A0A6G6A825;F6KD02;F6KD01...             PTEN   
4                              Q99805;A0A024QYR8;B3KSG9           TM9SF2   
...                                                 ...              ...   
8536                                             X5D7P8            STK39   
8537                                             X5D8X9          CNTNAP2   
8538  X5DQV1;X5DNI1;B3KV96;E9PD68;B3KXQ5;Q14194;B3KT...            CRMP1   
8539                                             X5DQZ7             GPX1   
8540                                             X6RHX1            SCYL3   

        sample                                                              \
     12-SEC61B     12-WT  13-RAB7A  12-G3BP1 12-TOMM20  12-LAMP1  13-RAB14   
0    -0.139405  0.497267  0.634090  0.208694 -0.654515 -0.266874  0.641974   
1    -0.631700  1.425600 -1.357550  0.621600 -2.248450 -0.588500 -0.984800   
2     4.080419 -3.924100  4.503702 -4.707100 -1.491024  3.421906  4.320300   
3    -0.874511  1.291439 -0.510095 -0.198848 -0.365514  0.959333 -0.412718   
4     7.104435 -6.935046  8.938121 -5.933239 -4.523732  5.796589  8.061546   
...        ...       ...       ...       ...       ...       ...       ...   
8536  0.967365 -0.679866  1.069567  0.137632  1.616812  0.362894  0.546989   
8537 -0.021208  0.374195  0.625338 -0.771679 -0.213794  0.194316  0.356751   
8538  0.491496 -0.357358 -0.820043  0.044803  0.351342  0.040040  1.146720   
8539  0.184980  0.766431  1.318104  0.794309  2.120314 -0.876673  0.025663   
8540  0.105110 -0.705130 -0.164388  0.293702 -0.494049  0.083745  0.466224   

      ...                                                              \
      ...   11-EEA1 11-GPR107 11-CEP350 09-ATG101   09-PEX3 11-SEC31A   
0     ... -0.681700 -0.615860  0.160560  3.270767 -0.086078 -1.159713   
1     ... -1.618800 -2.043600  0.895300 -0.728450 -0.496950  0.979850   
2     ... -1.672800  5.885700  0.397300  2.393000 -0.092300  1.688300   
3     ... -1.261271 -0.394072  0.064974  0.978763  0.488340 -0.652074   
4     ...  2.055000  5.827450 -0.438800 -0.412350  0.016550  1.179800   
...   ...       ...       ...       ...       ...       ...       ...   
8536  ...  0.023261 -0.641255  0.428594 -1.117751 -0.880850  0.993200   
8537  ...  1.104272 -0.473069  1.947072  0.136215  1.171869  1.749173   
8538  ...  0.722372 -0.688941 -0.044831 -0.041730  0.374553  0.029335   
8539  ... -1.142611  0.450772  0.828155 -0.488848  0.120523 -0.320115   
8540  ... -0.660514 -1.233455  0.931407 -0.077072 -0.049235  0.298556   

                                                      
       10-RTN4 NOC_cytosol NOC_organelle NOC_nuclear  
0    -0.158394    0.235748      0.241247    0.523005  
1    -1.899050    0.362391      0.286889    0.350721  
2     1.852200    0.054846      0.654148    0.291006  
3     0.656888    1.000000      0.000000    0.0000